# 第6章 卷积神经网络

## 6.1 从全连接层到卷积

### 练习 6.1.1

假设卷积层(6.3)覆盖的局部区域$\Delta = 0$。在这种情况下，证明卷积内核为每组通道独立地实现一个全连接层。

**解答：** 

局部区域$\Delta=0$ 表示卷积核的大小等于输入的大小。实际就是问，1×1的卷积核是否等价于全连接（参见本书7.3节：NiN网络结构）。因此，每个卷积核只能覆盖一个像素点，在这种情况下，卷积层的计算方式与全连接层非常相似。因为每个卷积核只能看到一个通道的信息，相当于每个卷积核只是一个全连接层的权重矩阵。 所以，卷积内核可以看作是每组通道独立地实现一个全连接层。每个卷积核都有自己的权重，每个输入通道都被独立处理，输出通道是各个输入通道的加权和。这种独立处理的方式有效地减少了权重的数量，从而降低了计算成本，并且能够提取出输入数据中的空间特征。

In [2]:
# 代码验证
import torch
import torch.nn as nn


class MyNet1(nn.Module):
    def __init__(self, linear1, linear2):
        super(MyNet1, self).__init__()
        self.linear1 = linear1
        self.linear2 = linear2

    def forward(self, X):
        return self.linear2(self.linear1(nn.Flatten()(X)))


class MyNet2(nn.Module):
    def __init__(self, linear, conv2d):
        super(MyNet2, self).__init__()
        self.linear = linear
        self.conv2d = conv2d

    def forward(self, X):
        X = self.linear(nn.Flatten()(X))
        X = X.reshape(X.shape[0], -1, 1, 1)
        X = nn.Flatten()(self.conv2d(X))
        return X


linear1 = nn.Linear(15, 10)
linear2 = nn.Linear(10, 5)
conv2d = nn.Conv2d(10, 5, 1)

linear2.weight = nn.Parameter(conv2d.weight.reshape(linear2.weight.shape))
linear2.bias = nn.Parameter(conv2d.bias)

net1 = MyNet1(linear1, linear2)
net2 = MyNet2(linear1, conv2d)

X = torch.randn(2, 3, 5)
# 两个结果实际存在一定的误差，直接print(net1(X) == net2(X))得到的结果不全是True
print(net1(X))
print(net2(X))

tensor([[0.1190, 0.2377, 0.1443, 0.1020, 0.0702],
        [0.1301, 0.2734, 0.1215, 0.0839, 0.1271]], grad_fn=<AddmmBackward0>)
tensor([[0.1190, 0.2377, 0.1443, 0.1020, 0.0702],
        [0.1301, 0.2734, 0.1215, 0.0839, 0.1271]],
       grad_fn=<ReshapeAliasBackward0>)


### 练习 6.1.2

为什么平移不变性可能也不是好主意呢？

**解答：** 

平移不变性可能会降低模型的准确性和泛化能力。对于某些任务，平移不变性并不是必须的特性。例如，对于图像分类任务，我们通常希望模型能够识别物体的位置和姿态，并根据这些信息对其进行分类。在这种情况下，平移不变性可能会降低模型的准确性，因为它忽略了物体的位置和姿态等重要信息。 其次，平移不变性可能会导致模型的泛化能力下降。

参考：[https://arxiv.org/pdf/1805.12177.pdf](https://arxiv.org/pdf/1805.12177.pdf)

### 练习 6.1.3

当从图像边界像素获取隐藏表示时，我们需要思考哪些问题？

**解答：** 

考虑是否填充padding，以及填充多大的padding的问题。可以使用torch.nn模块中的Pad函数对图像进行填充操作，以保证边界像素的信息完整。填充后还需要进行额外的处理，例如使用图像复制、填充、平滑等方法来获取隐藏表示。

### 练习 6.1.4

描述一个类似的音频卷积层的架构。

**解答：** 

一种基于卷积神经网络的音频特征生成方法，首先对声音信号进行预处理和离散傅里叶变换计算声音信号的幅度谱，形成二维谱图信号；然后搭建以上述二维谱图信号为输入的一维卷积神经网络并进行模型训练，得到特征生成器模型；最后对待测声音进行预处理和离散傅里叶变换得到二维谱图信号，并将其送入训练好的一维卷积神经网络，通过卷积网络计算，得到输出即为所要生成的音频特征，实现声音信号的音频特征生成。

### 练习 6.1.5

卷积层也适合于文本数据吗？为什么？

**解答：** 

卷积层也适合于文本数据。 在自然语言处理中，文本数据通常表示为词向量矩阵，其中每行代表一个词的向量表示。卷积层可以在这个矩阵上进行卷积操作，类似于图像卷积层中对图像进行卷积操作。 在卷积层中，卷积核会在输入矩阵上进行滑动窗口计算，输出一个新的特征矩阵。在文本数据中，这个特征矩阵可以看作是对输入文本的不同n-gram特征的提取。例如，一个大小为3的卷积核可以提取出输入文本中每个长度为3的n-gram特征。这些特征可以用于后续的分类或者回归任务。 此外，卷积层还可以与循环神经网络（RNN）结合使用，形成卷积神经网络（CNN）和循环神经网络（RNN）的混合模型。这种模型可以同时捕捉文本中的局部特征和全局特征，提高模型的性能。 因此，卷积层适用于文本数据，可以对文本数据进行卷积操作，提取出不同n-gram特征，并且可以与RNN结合使用，提高模型的性能。

### 练习 6.1.6

证明在式(6.6)中，$f * g = g * f$。

**解答：** 

通过式(6.6)的定义，我们可以得到：

$$(f * g)(x) = \int_{-\infty}^{\infty}f(y)g(x-y)dy$$

$$(g * f)(x) = \int_{-\infty}^{\infty}g(y)f(x-y)dy$$

要证明$f * g = g * f$，即证明：

$$\int_{-\infty}^{\infty}f(y)g(x-y)dy = \int_{-\infty}^{\infty}g(y)f(x-y)dy$$

为了证明上式成立，我们将其中一个积分的变量名改为$t=x-y$，则有：

$$\int_{-\infty}^{\infty}f(y)g(x-y)dy = \int_{-\infty}^{\infty}f(x-t)g(t)dt$$

再将这个式子代回式(6.6)中：

$$(f * g)(x) = \int_{-\infty}^{\infty}f(x-t)g(t)dt$$

对比式(6.6)和上面的式子，可以发现它们的形式是完全一样的，只是积分变量名不同而已。因此，我们可以得到：

$$(f * g)(x) = \int_{-\infty}^{\infty}f(y)g(x-y)dy = \int_{-\infty}^{\infty}g(y)f(x-y)dy = (g * f)(x)$$

因此，$f * g = g * f$，证毕。



## 6.2 图像卷积

### 练习 6.2.1  

构建一个具有对角线边缘的图像`X`。
1. 如果将本节中举例的卷积核`K`应用于`X`，会发生什么情况？
1. 如果转置`X`会发生什么？
1. 如果转置`K`会发生什么？

**解答：** 

1.在对角线处有分别为1和-1的数据，其他区域都为0。

In [3]:
import torch
from torch import nn
from d2l import torch as d2l

def corr2d(X, K):  #@save
    """计算二维互相关运算"""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

# 如果将本节中举例的卷积核K应用于X，会发生什么情况？
X = torch.eye(8)
K = torch.tensor([[1.0, -1.0]])
Y = corr2d(X, K)
print(Y)

tensor([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [-1.,  1.,  0.,  0.,  0.,  0.,  0.],
        [ 0., -1.,  1.,  0.,  0.,  0.,  0.],
        [ 0.,  0., -1.,  1.,  0.,  0.,  0.],
        [ 0.,  0.,  0., -1.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  0., -1.,  1.,  0.],
        [ 0.,  0.,  0.,  0.,  0., -1.,  1.],
        [ 0.,  0.,  0.,  0.,  0.,  0., -1.]])


2.转置后结果不变。

In [4]:
Y = corr2d(X.T, K)
X, Y

(tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1.]]),
 tensor([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.],
         [-1.,  1.,  0.,  0.,  0.,  0.,  0.],
         [ 0., -1.,  1.,  0.,  0.,  0.,  0.],
         [ 0.,  0., -1.,  1.,  0.,  0.,  0.],
         [ 0.,  0.,  0., -1.,  1.,  0.,  0.],
         [ 0.,  0.,  0.,  0., -1.,  1.,  0.],
         [ 0.,  0.,  0.,  0.,  0., -1.,  1.],
         [ 0.,  0.,  0.,  0.,  0.,  0., -1.]]))

3.K转置后，结果也转置了 

In [5]:
Y = corr2d(X, K.T)
X, Y

(tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1.]]),
 tensor([[ 1., -1.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  1., -1.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  1., -1.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  1., -1.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  1., -1.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  1., -1.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  1., -1.]]))

### 练习 6.2.2 

在我们创建的`Conv2D`自动求导时，有什么错误消息？

**解答：** 

错误信息：The size of tensor a (0) must match the size of tensor b (7) at non-singleton dimension 3
会提示维度不对称的错误信息，因为torch提供的二维卷积层是nn.Conv2d() 采用的是四维输入和输出格式（批量大小、通道、高度、宽度）,而我们自定义的仅仅是二维的。

### 练习 6.2.3

如何通过改变输入张量和卷积核张量，将互相关运算表示为矩阵乘法？

**解答：** 

题目的意思应该是如何通过矩阵乘法得到 互相关（卷积）运算

In [6]:
def conv2d_by_mul(X, K):
    # 获取卷积核大小
    h, w = K.shape
    # 计算输出图像大小
    outh = X.shape[0] - h + 1
    outw = X.shape[1] - w + 1
    # 调整卷积核形状以便做乘法
    K = K.reshape(-1, 1)
    # 将输入图像切成卷积核大小的块，打平成一维，存放在列表 Y 中
    Y = []
    for i in range(outh):
        for j in range(outw):
            Y.append(X[i:i + h, j:j + w].reshape(-1))
    # 将列表 Y 转为张量，每行代表一块的打平结果
    Y = torch.stack(Y, 0)
    # 用矩阵乘法表示互相关运算
    res = (torch.matmul(Y, K)).reshape(outh, outw)
    # 返回输出结果
    return res

### 练习 6.2.4

手工设计一些卷积核。
1. 二阶导数的核的形式是什么？
1. 积分的核的形式是什么？
1. 得到$d$次导数的最小核的大小是多少？

**解答：** 

二阶导数的核的形式是：

$$\begin{bmatrix}-1 & 2 & -1\end{bmatrix}$$

积分的核的形式是：

$$\begin{bmatrix}1 & 1 & 1 & \cdots & 1\end{bmatrix}$$

得到 𝑑 次导数的最小核的大小是 $d+1$。例如，一阶导数的最小核大小为 $2$，二阶导数的最小核大小为 $3$，三阶导数的最小核大小为 $4$，以此类推。


## 6.3 填充和步幅 

### 练习 6.3.1  

对于本节中的最后一个示例，计算其输出形状，以查看它是否与实验结果一致。

**解答：** 

$$out_{shape}=\lfloor(n_h-k_h+p_h+s_h)/s_h\rfloor \times \lfloor(n_w-k_w+p_w+s_w)/s_w\rfloor.$$

X.shape = [8, 8]

out_shape = [(8-3+0+3)/3, (8-5+1+4)/4] = [2.67, 2],向下取整，所以为[2, 2]



### 练习 6.3.2

在本节中的实验中，试一试其他填充和步幅组合。

**解答：** 

略。

### 练习 6.3.3

对于音频信号，步幅$2$说明什么？

**解答：** 

对于音频信号而言，步幅为2就是以2为周期对信号进行采样计算。

### 练习 6.3.4

步幅大于$1$的计算优势是什么？

**解答：** 

减小计算量， 减小内存占用， 提高模型的泛化能力。

## 6.4 多输入多输出通道 

### 练习 6.4.1

假设我们有两个卷积核，大小分别为$k_1$和$k_2$（中间没有非线性激活函数）。
1. 证明运算可以用单次卷积来表示。
1. 这个等效的单个卷积核的维数是多少呢？
1. 反之亦然吗？

**解答：** 

1. 假设输入的图像大小为W×H，设卷积核1的大小为 k1，卷积核2的大小为 k2，它们分别作用于输入矩阵 x，得到的输出矩阵分别为 y1 和 y2。则可以将 y1 与 y2 的每一个元素相加，得到最终输出矩阵 y。即：y[i][j] = y1[i][j] + y2[i][j]。可以将两个卷积核的大小相加，得到一个新的卷积核大小为(k1+k2-1)×(k1+k2-1)。然后可以将这个新的卷积核应用于输入图像，得到一个输出图像。这个输出图像的大小为(W-k1-k2+2)×(H-k1-k2+2)。
1. 可以使用一个大小为$(k1+k2-1)×(k1+k2-1)$的卷积核来表示这两个卷积核的运算。
1. 反之亦然。如果有一个大小为k1+k2-1的卷积核，可以将其分解为两个大小分别为k1和k2的卷积核。这两个卷积核之间没有非线性激活函数，所以它们的运算可以被视为一个单独的卷积核。

### 练习 6.4.2

假设输入为$c_i\times h\times w$，卷积核大小为$c_o\times c_i\times k_h\times k_w$，填充为$(p_h, p_w)$，步幅为$(s_h, s_w)$。
1. 前向传播的计算成本（乘法和加法）是多少？
1. 内存占用是多少？
1. 反向传播的内存占用是多少？
1. 反向传播的计算成本是多少？

**解答：** 

1. $flops_{forward} = c_i \times c_o \times k_h \times k_w \times \frac{h-p_h}{s_h} \times \frac{w-p_w}{s_w}$
1. 略
1. 略
1. $flops_{backward} = c_i \times c_o \times k_h \times k_w \times \frac{h-p_h}{s_h} \times \frac{w-p_w}{s_w} \times 2$

### 练习 6.4.3

如果我们将输入通道$c_i$和输出通道$c_o$的数量加倍，计算数量会增加多少？如果我们把填充数量翻一番会怎么样？

**解答：** 

如果我们将输入通道c_i和输出通道c_o的数量加倍，计算数量会增加4倍。如果我们把填充数量翻一番，计算数量会增加2倍。

### 练习 6.4.4

如果卷积核的高度和宽度是$k_h=k_w=1$，前向传播的计算复杂度是多少？

**解答：**

$$flops = c_i \times c_o \times \frac{h-p_h}{s_h} \times \frac{w-p_w}{s_w}$$

### 练习 6.4.5

本节最后一个示例中的变量`Y1`和`Y2`是否完全相同？为什么？

**解答：**

浮点数计算有误差，因而两者不完全相同。

### 练习 6.4.6

当卷积窗口不是$1\times 1$时，如何使用矩阵乘法实现卷积？

**解答：** 

可以将输入张量和卷积核张量分别展开为二维矩阵，然后对这两个矩阵进行乘法运算，得到的结果再变换为输出张量。

## 6.5 汇聚层 

### 练习 6.5.1

尝试将平均汇聚层作为卷积层的特殊情况实现。

**解答：** 

In [12]:
import torch.nn as nn 
import torch.nn.functional as F

class Net(nn.Module): 
    def init(self): 
      super(Net, self).init() 
      self.conv1 = nn.Conv2d(1, 6, 5) 
      self.pool = nn.Conv2d(6, 6, 5) 
      # 平均池化层 
      self.conv2 = nn.Conv2d(6, 16, 5) 
      self.fc1 = nn.Linear(16 * 5 * 5, 120) 
      self.fc2 = nn.Linear(120, 84) 
      self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.avg_pool2d(x, (2, 2)) # 平均池化层
        x = F.relu(self.conv2(x))
        x = F.avg_pool2d(x, (2, 2)) # 平均池化层
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

### 练习 6.5.2

尝试将最大汇聚层作为卷积层的特殊情况实现。

**解答：** 

In [11]:
import torch.nn as nn 
import torch.nn.functional as F

class Net(nn.Module): 
    def init(self): 
        super(Net, self).init() 
        self.conv1 = nn.Conv2d(1, 6, 5) 
        self.pool = nn.Conv2d(6, 6, 5) 
        # 最大池化层 
        self.conv2 = nn.Conv2d(6, 16, 5) 
        self.fc1 = nn.Linear(16 * 5 * 5, 120) 
        self.fc2 = nn.Linear(120, 84) 
        self.fc3 = nn.Linear(84, 10)  
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, (2, 2)) # 最大池化层
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, (2, 2)) # 最大池化层
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

### 练习 6.5.3

假设汇聚层的输入大小为$c\times h\times w$，则汇聚窗口的形状为$p_h\times p_w$，填充为$(p_h, p_w)$，步幅为$(s_h, s_w)$。这个汇聚层的计算成本是多少？

**解答：** 

$$\frac{c \times h \times w \times p_h \times p_w}{s_h \times s_w}$$

### 练习 6.5.4

为什么最大汇聚层和平均汇聚层的工作方式不同？

**解答：** 

最大池化层和平均池化层的工作方式不同，因为它们使用不同的池化方法。最大池化层将输入张量分成不重叠的区域，并在每个区域中选择最大值。平均池化层将输入张量分成不重叠的区域，并计算每个区域的平均值。这些方法的主要区别在于它们如何处理输入张量中的信息。最大池化层通常用于提取输入张量中的显著特征，而平均池化层通常用于减少输入张量的大小并提高模型的计算效率。

### 练习 6.5.5

我们是否需要最小汇聚层？可以用已知函数替换它吗？

**解答：** 

In [13]:
import torch.nn.functional as F

def min_pool2d(x, kernel_size, stride=None, padding=0, dilation=1, ceil_mode=False): 
    neg_x = -x 
    neg_min_pool = F.max_pool2d(neg_x, kernel_size, stride=stride, padding=padding, dilation=dilation, ceil_mode=ceil_mode) 
    min_pool = -neg_min_pool 
    return min_pool

### 练习 6.5.6

除了平均汇聚层和最大汇聚层，是否有其它函数可以考虑（提示：回想一下`softmax`）？为什么它不流行？

**解答：** 

## 6.6 卷积神经网络（LeNet） 

### 练习 6.6.1

将平均汇聚层替换为最大汇聚层，会发生什么？

**解答：** 

输出更大，梯度更大，训练更容易（AlexNet改进的方式之一）

### 练习 6.6.2

尝试构建一个基于LeNet的更复杂的网络，以提高其准确性。
1. 调整卷积窗口大小。
1. 调整输出通道的数量。
1. 调整激活函数（如ReLU）。
1. 调整卷积层的数量。
1. 调整全连接层的数量。
1. 调整学习率和其他训练细节（例如，初始化和轮数）。

**解答：** 

1. `nn.Conv2d(1, 6, kernel_size = 7)`
2. `nn.Conv2d(1, 10, kernel_size = 5)`
3. `nn.Conv2d(1, 6, kernel_size = 5).ReLU()`或者直接将`nn.Sigmoid()`改为`nn.ReLU()`
4. 添加`conv3`为`nn.Cov2d(16, 120, kernel_size = 5). ReLU()`
5. 添加`nn.Linear(84, 20)`并将`nn.Linear(84, 10)`替换为`nn.Linear(20, 10)`，添加`nn.Sigmoid()`
6. `lr, num_epochs = 0.1, 50`

### 练习 6.6.3

在MNIST数据集上尝试以上改进的网络。

**解答：** 

### 练习 6.6.4

显示不同输入（例如毛衣和外套）时，LeNet第一层和第二层的激活值。

**解答：** 

In [15]:
import torch
import torch.nn as nn
from torch.autograd import Variable

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = LeNet()
input_data = Variable(torch.randn(1, 3, 32, 32))
output_data = model(input_data)

# 计算第一层和第二层的激活值
x1 = model.conv1(input_data)
x2 = model.conv2(model.pool(F.relu(x1)))

print("第一层的激活值：", x1)
print("第二层的激活值：", x2)